In [1]:
import os

import basd
import numpy as np
import numpy.ma as ma
import pandas as pd
import xarray as xr

### Exploring the `xarray` package

   - Creating two `DataArray`s, affectively dataframes with extra attributes
        - One for precipitation and one for temperature at 4 grid cells
   - Adding these two `DataArray`s to a `Dataset`, more or less a dictionary for these data

In [2]:
# Precipitation data array
pr = xr.DataArray( np.random.rand(2,2), dims=('lat','lon'), coords={'lat':[35,10]})
pr.attrs['long_name'] = 'precipitation'
pr.attrs['units'] = 'inches'
pr.attrs['description'] = 'mean precipitation of grid cell, daily'

# Temperature data array
tas = xr.DataArray( np.random.rand(2,2), dims=('lat','lon'), coords={'lat':[35,10]})
tas.attrs['long_name'] = 'precipitation'
tas.attrs['units'] = 'inches'
tas.attrs['description'] = 'mean precipitation of grid cell, daily'

# Dataset of the two above data arrays
dataset = xr.Dataset(dict(pr=pr, tas=tas))

#### Reading in a NetCDF file

Reading in observational precipitation data from 1979-2014 supplied in the data directory.

In [3]:
pr_obs_hist = xr.open_dataset('../basd/data/pr_obs-hist_coarse_1979-2014.nc')
pr_sim_hist = xr.open_dataset('../basd/data/pr_sim-hist_coarse_1979-2014.nc')
pr_sim_fut = xr.open_dataset('../basd/data/pr_sim-fut_coarse_2065-2100.nc')

## Running a Bias Adjustment

### Initializing our first `Adjustment` object

Here we are providing the three required data arrays and the variable name 'pr'. For precipitation, a variable that will be bounded below, we want to input some additional parameters:

- Lower bound = 0
- Lower threshold = 0.001
- Trend preservation = mixed
- Distribution = gamma

Otherwise, we are keeping the default parameter values for the bias adjustment

In [7]:
ba = basd.Adjustment(pr_obs_hist, pr_sim_hist, pr_sim_fut, 'pr',
                     lower_bound=0, lower_threshold=0.01, trend_preservation='mixed',
                     distribution='gamma')

#### Initializing running window mode

Here we want to scrape the month, year, or day of the year, for each observation. We will end up using some but not all of this info depending on the adjustment method chosen by the parameters.

In [18]:
# Scraping the time from the data and turning into pandas date time array
dates_obs_hist = pd.DatetimeIndex(ba.obs_hist['time'].values)
dates_sim_hist = pd.DatetimeIndex(ba.sim_hist['time'].values)
dates_sim_fut = pd.DatetimeIndex(ba.sim_fut['time'].values)

# Getting the month for each observation
month_numbers = {
    'obs_hist': dates_obs_hist.month,
    'sim_hist': dates_sim_hist.month,
    'sim_fut': dates_sim_fut.month
}

# Getting the year for each observation
years = {
    'obs_hist': dates_obs_hist.year,
    'sim_hist': dates_sim_hist.year,
    'sim_fut': dates_sim_fut.year
}

# Getting the day of the year for each observation
days = {
    'obs_hist': dates_obs_hist.day,
    'sim_hist': dates_sim_hist.day,
    'sim_fut': dates_sim_fut.day
}
